<a href="https://colab.research.google.com/github/blaze2004/Flutter-Supabase-OAuth/blob/main/apps/inference/Llama_3_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq -U xformers transformers datasets accelerate peft trl bitsandbytes wandb triton huggingface_hub
!pip install "unsloth[colab-new] @ git+https://GitHub.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import gc
import os
import json

import torch
import wandb
from datasets import load_dataset
from google.colab import userdata
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments
)
from trl import SFTTrainer, setup_chat_format
from unsloth import FastLanguageModel

# Model
base_model = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
# base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "Counselling-llama-3.1-8B-Instruct"

# Defined in the secrets tab in Google Colab
wb_token = userdata.get('wandb')
wandb.login(key=wb_token)
os.environ["WANDB_PROJECT"] = "shanti"

# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Defining the configuration for the base model, LoRA and training
config = {
    "hugging_face_username":"blaze2004",
    "model_config": {
        "base_model":"unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", # The base model
        "finetuned_model":"Counselling-llama-3.1-8B-Instruct", # The fine-tuned model
        "max_seq_length": None, # The maximum sequence length
        "dtype":torch.float16, # The data type
        "load_in_4bit": True, # Load the model in 4-bit
    },
    "lora_config": {
      "r": 16, # The number of LoRA layers 8, 16, 32, 64
      "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # The target modules
      "lora_alpha":16, # The alpha value for LoRA
      "lora_dropout":0, # The dropout value for LoRA
      "bias":"none", # The bias for LoRA
      "use_gradient_checkpointing":True, # Use gradient checkpointing
      "use_rslora":False, # Use RSLora
      "use_dora":False, # Use DoRa
      "loftq_config":None # The LoFTQ configuration
    },
    "training_dataset":{
        "name":"blaze2004/llama-counselling", # The dataset name(huggingface/datasets)
        "input_field":"text", # The input field
    },
    "training_config": {
        "per_device_train_batch_size": 2, # The batch size
        "gradient_accumulation_steps": 4, # The gradient accumulation steps
        "warmup_steps": 5, # The warmup steps
        "max_steps":0, # The maximum steps (0 if the epochs are defined)
        "num_train_epochs": 1, # The number of training epochs(0 if the maximum steps are defined)
        "learning_rate": 2e-4, # The learning rate
        "fp16": not torch.cuda.is_bf16_supported(), # The fp16
        "bf16": torch.cuda.is_bf16_supported(), # The bf16
        "logging_steps": 1, # The logging steps
        "optim" :"adamw_8bit", # The optimizer
        "weight_decay" : 0.01,  # The weight decay
        "lr_scheduler_type": "linear", # The learning rate scheduler
        "seed" : 42, # The seed
        "output_dir" : "outputs", # The output directory
    }
}

In [ ]:
# Loading the model and the tokinizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config.get("model_config").get("base_model"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dtype = config.get("model_config").get("dtype"),
    load_in_4bit = config.get("model_config").get("load_in_4bit"),
    device_map="auto"
)

# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r = config.get("lora_config").get("r"),
    target_modules = config.get("lora_config").get("target_modules"),
    lora_alpha = config.get("lora_config").get("lora_alpha"),
    lora_dropout = config.get("lora_config").get("lora_dropout"),
    bias = config.get("lora_config").get("bias"),
    use_gradient_checkpointing = config.get("lora_config").get("use_gradient_checkpointing"),
    random_state = 42,
    use_rslora = config.get("lora_config").get("use_rslora"),
    use_dora = config.get("lora_config").get("use_dora"),
    loftq_config = config.get("lora_config").get("loftq_config"),
)

# Loading the training dataset
dataset_name = "blaze2004/llama-counselling"
dataset = load_dataset(dataset_name)

def format_chat_template(row):
    row["text"] = tokenizer.apply_chat_template(row["text"], tokenize=False)
    return row

dataset_train = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)



# Setting up the trainer for the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train["train"],
    eval_dataset = dataset_train["test"],
    dataset_text_field = config.get("training_dataset").get("input_field"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = config.get("training_config").get("per_device_train_batch_size"),
        gradient_accumulation_steps = config.get("training_config").get("gradient_accumulation_steps"),
        warmup_steps = config.get("training_config").get("warmup_steps"),
        max_steps = config.get("training_config").get("max_steps"),
        num_train_epochs= config.get("training_config").get("num_train_epochs"),
        learning_rate = config.get("training_config").get("learning_rate"),
        fp16 = config.get("training_config").get("fp16"),
        bf16 = config.get("training_config").get("bf16"),
        logging_steps = config.get("training_config").get("logging_steps"),
        optim = config.get("training_config").get("optim"),
        weight_decay = config.get("training_config").get("weight_decay"),
        lr_scheduler_type = config.get("training_config").get("lr_scheduler_type"),
        seed = 42,
        output_dir = config.get("training_config").get("output_dir"),
        report_to="wandb",
        save_strategy = "steps",
        save_steps = 5,
        eval_steps=100,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                 # Perform evaluation at the end of training
        do_predict=True,
        hub_model_id=config.get("hugging_face_username")+"/"+config.get("model_config").get("finetuned_model"),
        push_to_hub=True,
        hub_private_repo=True,
        resume_from_checkpoint=True,
        save_total_limit=2,
        hub_strategy="checkpoint",
    ),
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

Map (num_proc=2):   0%|          | 0/25875 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/21 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/42 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/25875 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/42 [00:00<?, ? examples/s]

In [ ]:
# Memory statistics before training
gpu_statistics = torch.cuda.get_device_properties(0)
reserved_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
max_memory = round(gpu_statistics.total_memory / 1024**3, 2)
print(f"Reserved Memory: {reserved_memory}GB")
print(f"Max Memory: {max_memory}GB")

Reserved Memory: 5.98GB
Max Memory: 14.75GB


# Fetch checkpoint to resume training

In [ ]:
!git config --global credential.helper store

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import snapshot_download
import json

repo_id = config.get("hugging_face_username")+"/"+config.get("model_config").get("finetuned_model")
local_dir = "./outputs"

snapshot_download(repo_id=repo_id, local_dir=local_dir)
with open("./outputs/last-checkpoint/trainer_state.json", "r") as f:
    last_checkpoint = json.loads(f.read())
    last_checkpoint_step = last_checkpoint.get("global_step")

os.rename("./outputs/last-checkpoint", "./outputs/checkpoint-"+str(last_checkpoint_step))

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

last-checkpoint/README.md:   0%|          | 0.00/5.12k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

# Train

In [ ]:
def contains_subdirectory(directory):
    for entry in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, entry)):
            return True
    return False

In [ ]:
if contains_subdirectory(f"{config.get('training_config').get('output_dir')}"):
  trainer_stats = trainer.train(resume_from_checkpoint=True)
else:
  trainer_stats = trainer.train()
trainer.save_model(new_model)

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

wandb: Currently logged in as: blaze2004 (visual-brahma). Use `wandb login --relogin` to force relogin


Step,Training Loss


training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

In [ ]:
# Memory statistics after training
used_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
used_memory_lora = round(used_memory - reserved_memory, 2)
used_memory_persentage = round((used_memory / max_memory) * 100, 2)
used_memory_lora_persentage = round((used_memory_lora / max_memory) * 100, 2)
print(f"Used Memory: {used_memory}GB ({used_memory_persentage}%)")
print(f"Used Memory for training(fine-tuning) LoRA: {used_memory_lora}GB ({used_memory_lora_persentage}%)")

Used Memory: 6.14GB (41.63%)
Used Memory for training(fine-tuning) LoRA: 0.16GB (1.08%)


In [ ]:
# Saving the trainer stats
with open("trainer_stats.json", "w") as f:
    json.dump(trainer_stats, f, indent=4)

In [ ]:
# # Locally saving the model and pushing it to the Hugging Face Hub (only LoRA adapters)
# model.save_pretrained(config.get("model_config").get("finetuned_model"))
model.push_to_hub(config.get("model_config").get("finetuned_model"), tokenizer = tokenizer)

README.md:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/Counselling-llama-3.1-8B-Instruct


In [ ]:
# Saving the model using merged_16bit(float16), merged_4bit(int4) or quantization options(q8_0, q4_k_m, q5_k_m)...

# model.save_pretrained_merged(config.get("model_config").get("finetuned_model"), tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged(config.get("model_config").get("finetuned_model"), tokenizer, save_method = "merged_16bit")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.43 out of 12.67 RAM for saving.


100%|██████████| 32/32 [00:44<00:00,  1.39s/it]


Unsloth: Saving to organization with address blaze2004/Counselling-llama-3.1-8B-Instruct
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving to organization with address blaze2004/Counselling-llama-3.1-8B-Instruct
Unsloth: Saving blaze2004/Counselling-llama-3.1-8B-Instruct/pytorch_model-00001-of-00004.bin...
Unsloth: Saving blaze2004/Counselling-llama-3.1-8B-Instruct/pytorch_model-00002-of-00004.bin...
Unsloth: Saving blaze2004/Counselling-llama-3.1-8B-Instruct/pytorch_model-00003-of-00004.bin...
Unsloth: Saving blaze2004/Counselling-llama-3.1-8B-Instruct/pytorch_model-00004-of-00004.bin...
Unsloth: Uploading all files... Please wait...


pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/Counselling-llama-3.1-8B-Instruct


Exception ignored in: <function _xla_gc_callback at 0x7c21a6304e50>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


RuntimeError: Unsloth: Merging into 4bit will cause your model to lose accuracy if you plan
to merge to GGUF or others later on. I suggest you to do this as a final step
if you're planning to do multiple saves.
If you are certain, change `save_method` to `merged_4bit_forced`.

In [ ]:
model.push_to_hub_gguf(
        config.get("model_config").get("finetuned_model"),
        tokenizer,
        quantization_method = ["q8_0", "q5_k_m", "q4_k_m"]
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.12 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:52<00:00,  3.52s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving Counselling-llama-3.1-8B-Instruct/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Counselling-llama-3.1-8B-Instruct/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Counselling-llama-3.1-8B-Instruct/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Counselling-llama-3.1-8B-Instruct/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0', 'q5_k_m', 'q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at Counselling-llama-3.1-8B-Instruct into f16 GGUF format.
The output location will be ./Counselling-llama-3.1-8B-Instruct/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: Counselling-llama-3.1-8B-Instruct
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch

...

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/blaze2004/Counselling-llama-3.1-8B-Instruct
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/blaze2004/Counselling-llama-3.1-8B-Instruct


In [ ]:
model.push_to_hub_merged(config.get("model_config").get("finetuned_model"), tokenizer, save_method = "merged_4bit")

In [ ]:
# Loading the fine-tuned model and the tokenizer for inference
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = new_model,
        max_seq_length = config.get("model_config").get("max_seq_length"),
        dtype = config.get("model_config").get("dtype"),
        load_in_4bit = config.get("model_config").get("load_in_4bit"),
    )

# Using FastLanguageModel for fast inference
FastLanguageModel.for_inference(model)

# Tokenizing the input and generating the output
inputs = tokenizer(
[
    "<|start_header_id|>system<|end_header_id|> Answer the question truthfully, you are a medical professional.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: Can you provide an overview of the lung's squamous cell carcinoma?<|eot_id|>"
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens = True)